In [ ]:
#CAMBIARE RUN TIME

In [ ]:
!pip install -U bitsandbytes

riavviare runtime dopo l installazione di bitsandbytes

In [ ]:
# --- Blocco 1: Setup Iniziale per Mistral-7B-Instruct-v0.2 su GPU ---

import os
import json
import re
import threading
import pandas as pd
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig
import torch

# 1. Montare Google Drive
try:
    drive.mount('/content/drive', force_remount=True) # force_remount può essere utile
    print("Google Drive montato con successo!")
except Exception as e:
    print(f"Errore durante il montaggio di Google Drive: {e}")
    raise SystemExit("Impossibile montare Google Drive. Verifica i permessi.")

# 2. Definire il percorso del modello salvato su Drive e configura la quantizzazione

model_path_on_drive = "/content/drive/MyDrive/Mistral-7B-Instruct-v0.2"

# Controllo disponibilità CUDA (GPU)
if not torch.cuda.is_available():
    print("ERRORE: GPU non disponibile. Assicurati di aver selezionato un runtime con GPU (Runtime -> Change runtime type).")
    raise SystemExit("GPU non trovata.")

# Configurazione per la quantizzazione a 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16 # Cambiato a float16 come suggerito per compatibilità GPU
)
print(f"Tentativo di caricare il modello da: {model_path_on_drive} con quantizzazione a 4-bit su GPU.")

# 3. Carica Tokenizer e Modello su GPU
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path_on_drive)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    print("Tokenizer caricato e pad_token impostato.")

    model = AutoModelForCausalLM.from_pretrained(
        model_path_on_drive,
        quantization_config=bnb_config, # quantizzazione
        device_map="auto",  # accelerate gestisce il mapping sulla GPU
        torch_dtype=torch.float16 # Dovrebbe corrispondere a bnb_4bit_compute_dtype
                                   # anche qui provare torch.float16 se bfloat16 da problemi
    )
    model.eval() # Imposta il modello in modalità valutazione

    # Verifica dispositivo
    if hasattr(model, 'device'):
        print(f"Modello caricato. Dispositivo principale del modello: {model.device}")
        if len(list(model.parameters())) > 0:
            param_device = next(model.parameters()).device
            print(f"Parametri del modello allocati su: {param_device}")
            if "cuda" not in str(param_device):
                print("ATTENZIONE: Il modello potrebbe non essere sulla GPU come previsto!")
        else:
             print("ATTENZIONE: Modello senza parametri?")
    else:
        print("Modello caricato, ma l'attributo 'device' non è direttamente accessibile.")


except ImportError as e_imp:
    print(f"ImportError: {e_imp}")
    print("Questo errore spesso indica che 'bitsandbytes' non è installato correttamente o il kernel non è stato riavviato dopo l'installazione.")
    print("Prova ad aggiungere '!pip install -U bitsandbytes' all'inizio di questa cella, eseguila, POI RIAVVIA IL RUNTIME (Menu Runtime > Riavvia sessione) e riesegui questa cella.")
    raise SystemExit("Errore di importazione, probabilmente bitsandbytes.")
except Exception as e:
    print(f"Errore durante il caricamento del modello {model_path_on_drive} da Drive: {e}")
    import traceback
    traceback.print_exc()
    raise SystemExit("Impossibile caricare il modello da Drive.")

print("\n--- Setup GPU completato. 'model' e 'tokenizer' sono pronti. ---")

# --- Variabile globale per il prompt dell'utente ---
user_prompt_template = """
You are an expert analyst specializing in patents and scientific literature.
Your task is to read technical texts and distill their core technological concepts into clear, concise sentences.
You excel at identifying the main purpose of an invention or research (Function), explaining how it works (Solution), and stating where it can be applied (Application).
Your summaries are precise, scientifically accurate, and easily understandable to researchers and professionals.

Text:
{text}

Respond ONLY in the following JSON format and the response MUST start with {{ and end with }}:

{{
  "function": "<brief summary of the main purpose, in verb-object form>",
  "solution": "<brief technical explanation of how the purpose is achieved>",
  "application": "<brief summary of practical or industrial domains where it applies>"
}}
"""

In [ ]:
# --- Blocco 2: Definizione delle Funzioni di Elaborazione ---

# Verifica che le variabili dal setup siano presenti
if 'model' not in globals() or 'tokenizer' not in globals() or 'user_prompt_template' not in globals():
    print("ERRORE CRITICO: 'model', 'tokenizer', o 'user_prompt_template' non sono definiti.")
    print("Esegui prima la CELLA 1 (Setup Iniziale).")
    raise SystemExit("Setup del modello non completato.")

REQUIRED_KEYS_EXTRACT = {"function", "solution", "application"}
def is_valid_output_extract(output): # Usata dopo in extract_concepts
    if not isinstance(output, dict):
        return False
    return REQUIRED_KEYS_EXTRACT.issubset(output.keys())

save_lock = threading.Lock() # Per la scrittura su file thread-safe

def save_block_to_jsonl(results, path='results_patent.jsonl'):
    with save_lock:
        with open(path, 'a', encoding='utf-8') as f:
            for r in results:
                f.write(json.dumps(r, ensure_ascii=False) + '\n')

def get_already_processed_patent_ids(jsonl_path):
    if not os.path.exists(jsonl_path):
        return set()
    done = set()
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                obj = json.loads(line)
                if "patent_id" in obj:
                    done.add(str(obj["patent_id"]))
            except Exception:
                continue
    return done

def is_valid_final_output(result): # Usata dopo in process_batch prima di salvare
    if not isinstance(result, dict): return False
    return all(k in result and isinstance(result[k], str) for k in ["function", "solution", "application", "patent_id"])

# MODIFICA aggiunta per DEBUG: extract_concepts
def extract_concepts(text):
    global model, tokenizer, user_prompt_template # per assicurare l'uso delle variabili globali
    chat_messages = [{"role": "user", "content": user_prompt_template.format(text=text)}]
    try:
        prompt_for_llm = tokenizer.apply_chat_template(chat_messages, tokenize=False, add_generation_prompt=True)
    except Exception as e:
        print(f"DEBUG extract_concepts: Errore apply_chat_template: {e}")
        return None

    inputs = tokenizer(prompt_for_llm, return_tensors="pt", return_attention_mask=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id
        )

    generated_ids = outputs[0][inputs['input_ids'].shape[-1]:]
    decoded_output = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    #print(f"\nDEBUG decoded_output COMPLETO:\n{decoded_output}\n")

    json_match = re.search(r'\{[\s\S]*\}', decoded_output)
    #json_match = re.search(r'\{[\s\S]*?\}', decoded_output)
    cleaned_json_str = None
    if json_match:
        json_str_raw = json_match.group().strip()
        if json_str_raw.startswith("```json"):
            json_str_raw = json_str_raw[len("```json"):].strip()
        if json_str_raw.endswith("```"):
            json_str_raw = json_str_raw[:-len("```")].strip()
        cleaned_json_str = json_str_raw

    if cleaned_json_str:
        try:
            parsed_json = json.loads(cleaned_json_str)
            if is_valid_output_extract(parsed_json):
                print("DEBUG extract_concepts: JSON PARSATO E VALIDO (con function, solution, application).")
                return parsed_json
            else:
                print("DEBUG extract_concepts: JSON PARSATO MA NON VALIDO (mancano chiavi 'function', 'solution', o 'application').")
                print(f"DEBUG extract_concepts: Chiavi trovate: {list(parsed_json.keys())}")
                return None
        except json.JSONDecodeError as e:
            print(f"DEBUG extract_concepts: JSONDecodeError: '{e}'")
            print(f"DEBUG extract_concepts: Stringa JSON (pulita) che ha causato l'errore (primi 200): '{cleaned_json_str[:200]}...'")
            return None
    else:
        print("DEBUG extract_concepts: Nessun blocco JSON trovato nell'output dell'LLM con regex.")
        return None

def process_row(index, row):
    global model, tokenizer # Anche se non usate direttamente qui, extract_concepts le usa
    try:
        text_parts = []
        if "appln_title" in row and pd.notna(row["appln_title"]): text_parts.append(str(row["appln_title"]))
        if "appln_abstract" in row and pd.notna(row["appln_abstract"]): text_parts.append(str(row["appln_abstract"]))
        if not text_parts:
            print(f"DEBUG process_row: Riga {index} non ha titolo/abstract. Salto.")
            return None
        text = ". ".join(text_parts)

        result_from_llm = extract_concepts(text)
        if result_from_llm is None: return None

        if "patent" in row and pd.notna(row["patent"]):
            result_from_llm["patent_id"] = str(row["patent"])
        else:
            print(f"DEBUG process_row: Riga {index} manca 'patent_id' (colonna 'patent' nel df). Scartata.")
            return None
        return result_from_llm
    except Exception as e:
        print(f"DEBUG process_row: Errore imprevisto riga {index}: {e}")
        return None

def process_batch(current_batch_df, path_to_save_results):
    batch_results_list = []
    print(f"    DEBUG process_batch: Inizio elaborazione di {len(current_batch_df)} righe...")
    for original_index, row_data in current_batch_df.iterrows():
        single_row_result = process_row(original_index, row_data)
        if single_row_result is not None:
            batch_results_list.append(single_row_result)

    print(f"    DEBUG process_batch: Elaborazione batch completata. Ottenuti {len(batch_results_list)} risultati grezzi.")
    valid_results_for_saving = [r for r in batch_results_list if is_valid_final_output(r)]

    if len(valid_results_for_saving) > 0:
        if len(valid_results_for_saving) < len(batch_results_list):
            print(f"    DEBUG process_batch: ATTENZIONE - {len(batch_results_list) - len(valid_results_for_saving)} risultati grezzi non validi e scartati.")
        save_block_to_jsonl(valid_results_for_saving, path_to_save_results)
        print(f"    DEBUG process_batch: Batch salvato con {len(valid_results_for_saving)} risultati validi su {path_to_save_results}.")
    else:
        print(f"    DEBUG process_batch: Nessun risultato valido da salvare per questo batch.")
    return len(valid_results_for_saving)

print("--- Funzioni di elaborazione definite. ---")

In [ ]:
# prova con output che non stampa i debug ma la percentuale di completamento e che salva riga per riga senza aspettare la fine del batch
# --- Blocco 3: Esecuzione Principale con Salvataggio Riga per Riga e Barra di Progresso ---
from tqdm.auto import tqdm # Importa tqdm per la barra di progresso

# Verifica che le variabili e funzioni cruciali siano definite
if 'df' not in globals() and ('model' not in globals() or 'tokenizer' not in globals() or 'process_row' not in globals()):
    print("ERRORE CRITICO: DataFrame 'df' O ('model', 'tokenizer', 'process_row') non definiti.")
    print("Esegui prima la CELLA 1 (Setup) e la CELLA 2 (Definizione Funzioni). Poi carica 'df'.")
    # Tenta di caricare df se non definito
    try:
        df_path = r"/content/drive/MyDrive/data_giacomo.csv"
        print(f"Tentativo di caricare il DataFrame da: {df_path}")
        df = pd.read_csv(df_path)
        print(f"DataFrame caricato con {len(df)} righe.")
    except FileNotFoundError:
        print(f"ERRORE: File DataFrame '{df_path}' non trovato. Impossibile procedere.")
        raise SystemExit("DataFrame non trovato.")
    except Exception as e:
        print(f"ERRORE durante il caricamento del DataFrame: {e}")
        raise SystemExit("Errore caricamento DataFrame.")
elif 'df' not in globals(): # Se model, tokenizer, ecc. sono definiti ma df no
    try:
        df_path = r"/content/drive/MyDrive/data_giacomo.csv"
        print(f"Tentativo di caricare il DataFrame da: {df_path}")
        df = pd.read_csv(df_path)
        print(f"DataFrame caricato con {len(df)} righe.")
    except FileNotFoundError:
        print(f"ERRORE: File DataFrame '{df_path}' non trovato. Impossibile procedere.")
        raise SystemExit("DataFrame non trovato.")
    except Exception as e:
        print(f"ERRORE durante il caricamento del DataFrame: {e}")
        raise SystemExit("Errore caricamento DataFrame.")
else:
     print(f"DataFrame 'df' ({len(df)} righe), Setup modello e funzioni OK. Procedo.")


results_path = "/content/drive/MyDrive/results_patent_mistral7b_gpu.jsonl"
print(f"\nL'output verrà salvato riga per riga in: {results_path}")
print("Assicurati che le colonne 'patent', 'appln_title', 'appln_abstract' siano presenti nel DataFrame 'df'.")

NUMERO_RIGHE_DA_PROCESSARE_ORA = 400

done_patent_ids = get_already_processed_patent_ids(results_path)
print(f"Trovati {len(done_patent_ids)} patent_id già processati nel file di output.")

if "patent" not in df.columns:
    print("ERRORE: La colonna 'patent' non esiste nel DataFrame 'df'.")
else:
    df_all_new_rows = df[~df["patent"].astype(str).isin(done_patent_ids)]
    print(f"Numero di righe totali nel DataFrame 'df': {len(df)}")
    print(f"Numero di righe nuove (non ancora processate): {len(df_all_new_rows)}")

    if len(df_all_new_rows) == 0:
        print("Nessuna nuova riga da processare.")
    else:
        df_this_run = df_all_new_rows.head(NUMERO_RIGHE_DA_PROCESSARE_ORA)

        if len(df_this_run) == 0:
            print(f"Nessuna riga selezionata per questa esecuzione (limite: {NUMERO_RIGHE_DA_PROCESSARE_ORA} o nessuna riga nuova).")
        else:
            print(f"Verranno processate e salvate individualmente le prossime {len(df_this_run)} righe.")

            processed_count_in_this_run = 0
            # Itera sulle righe selezionate per barra di progresso
            for original_index, row_data in tqdm(df_this_run.iterrows(), total=len(df_this_run), desc="Processing rows"):
                try:
                    # Processa la singola riga
                    single_row_result = process_row(original_index, row_data)

                    if single_row_result is not None:
                        # Valida il risultato finale (include patent_id)
                        if is_valid_final_output(single_row_result):
                            # Salva il singolo risultato valido
                            save_block_to_jsonl([single_row_result], results_path)
                            processed_count_in_this_run += 1
                except Exception as e_row_proc:
                    print(f"ERRORE grave durante l'elaborazione della riga con indice originale {original_index}: {e_row_proc}")
                    # import traceback; traceback.print_exc() # Per debug più approfondito

            print(f"\n--- Elaborazione di {len(df_this_run)} righe richiesta completata. ---")
            print(f"Salvati {processed_count_in_this_run} risultati validi in questo run.")

    print(f"Controlla il file di output: {results_path}")

In [ ]:
# check per vedere se l output esiste:
!ls -l /content/drive/MyDrive/results_patent_mistral7b_gpu.jsonl
#!cat /content/drive/MyDrive/results_patent_mistral7b_gpu.jsonl # ATTENZIONE: stampa tutto il contenuto, usa con cautela se il file è grande

In [ ]:
# pe controllare quanti risultati contiene il mio json di output
import json
import os # Aggiunto per controllare se il file esiste
from google.colab import drive

# 1. Monta Google Drive
try:
    drive.mount('/content/drive', force_remount=True) # force_remount può essere utile
    print("Google Drive montato con successo!")
except Exception as e:
    print(f"Errore durante il montaggio di Google Drive: {e}")
    raise SystemExit("Impossibile montare Google Drive. Verifica i permessi.")

def analizza_file_jsonl(percorso_file_jsonl):
    """
    Analizza un file JSONL per contare le righe totali,
    gli oggetti JSON validi, e gli ID brevetto unici.
    """
    conteggio_righe_totali = 0
    conteggio_oggetti_json_validi = 0
    patent_ids_unici_nel_file = set()
    errori_parsing_json = 0
    righe_senza_patent_id = 0

    print(f"--- Analisi del file: {percorso_file_jsonl} ---")

    if not os.path.exists(percorso_file_jsonl):
        print(f"ERRORE: Il file specificato non è stato trovato: {percorso_file_jsonl}")
        return 0, 0, 0, 0, 0

    try:
        with open(percorso_file_jsonl, 'r', encoding='utf-8') as f:
            for i, riga in enumerate(f):
                conteggio_righe_totali += 1
                try:
                    obj = json.loads(riga)
                    conteggio_oggetti_json_validi += 1
                    if "patent_id" in obj and obj["patent_id"] is not None:
                        patent_ids_unici_nel_file.add(str(obj["patent_id"]))
                    else:
                        righe_senza_patent_id += 1
                        # print(f"Riga {i+1} non contiene 'patent_id' o è None: {riga.strip()[:100]}...") # Debug opzionale
                except json.JSONDecodeError:
                    errori_parsing_json += 1
                    # print(f"Riga {i+1} non è un JSON valido: {riga.strip()[:100]}...") # Debug opzionale

        print(f"Numero totale di righe nel file: {conteggio_righe_totali}")
        print(f"Numero di oggetti JSON validi parsati: {conteggio_oggetti_json_validi}")
        print(f"Numero di ID brevetto ('patent_id') UNICI trovati: {len(patent_ids_unici_nel_file)}")
        if righe_senza_patent_id > 0:
            print(f"ATTENZIONE: {righe_senza_patent_id} oggetti JSON validi non avevano una chiave 'patent_id' o era None.")
        if errori_parsing_json > 0:
            print(f"ATTENZIONE: {errori_parsing_json} righe non sono state parsate correttamente come JSON.")

        return conteggio_righe_totali, conteggio_oggetti_json_validi, len(patent_ids_unici_nel_file)

    except Exception as e:
        print(f"Si è verificato un errore generale durante la lettura del file: {e}")
        return 0, 0, 0

# --- ESECUZIONE DELL'ANALISI ---
percorso_del_tuo_output_jsonl = "/content/drive/MyDrive/results_patent_mistral7b_gpu.jsonl"
#funzione di analisi
tot_lines, valid_json_obj, unique_ids = analizza_file_jsonl(percorso_del_tuo_output_jsonl)

print("\n--- Riepilogo ---")
if tot_lines > 0:
    print(f"Il tuo file di output contiene {tot_lines} righe.")
    print(f"Di queste, {valid_json_obj} sono state lette come oggetti JSON validi.")
    print(f"All'interno di questi oggetti JSON validi, sono stati trovati {unique_ids} 'patent_id' unici.")
    print("\nQuesto numero di 'patent_id' unici è quello che viene usato per calcolare le 'righe nuove da processare'.")
else:
    if os.path.exists(percorso_del_tuo_output_jsonl):
        print("Il file di output sembra essere vuoto o illeggibile.")
    else:
        print("Il file di output non è stato trovato. Verifica il percorso.")